# 08 Quantum Defense

## Lesson Goals

1. Students will learn about various types of quantum errors.
2. Students will be able to construct quantum error correction codes.
3. Students will understand larger implications of correcting errors in quantum computing.
4. Students will learn about various attacks on quantum key distribution schemes.

## 08.01: Quantum Error Correction Codes

### 08.01.01: Introduction

Coding theory and cryptography are two most intricately connected subjects. In several cases, cryptosystems are simply error correction codes for which the decoding matrix forms the key. For example, the post-quantum encryption algorithm called McEllice cryptosystem is based on Goppa codes. There are numerous other examples where polynomial arithmetic is used for encryption and key management. Therefore, it is important for anyone learning cryptography (or quantum cryptography) to be familiar with the theory of quantum error correction codes. This lesson will provide a brief introduction to the topic in order to wet the appetite of the reader. For further details, the reader should refer to books mentioned in the preface of this unit.

Qubits are prone to errors during transit from sender to receiver. In classical communication, we only concern ourselves with bit flip errors where a transmitted 0 changes to 1 and vice versa. With qubits we have to theoretically worry about infinite number of possible errors because a qubit is a continuum state described by $\alpha|0\rangle+\beta|1\rangle$ with $\alpha$ and $\beta$ being complex numbers. An error in such a qubit would amount to $\alpha$ changing to some $\alpha'$ and $\beta$ changing to $\beta'$. For the longest time it was believed that such errors cannot be corrected because every value of $\alpha$ and $\beta$ is possible and, therefore, one cannot distinguish between a valid state and an invalid state. However, in 1995 Peter Shor proved everyone wrong by inventing quantum error correction and providing a 9-qubit coding scheme that can correct any arbitrary single qubit error. Later this code was refined to 7-qubit and 5-qubit schemes where the latter is the most efficient a single qubit error correction scheme can be.

Besides errors that can happen during qubits transit, a qubit can also suffer with errors because of imperfect implementation of hardware that creates these error correcting codes. Correcting such errors falls under the field of fault-tolerant quantum computing and will not concern us here.

### 08.01.02: Errors in Qubits

The two types of errors that immediately come to mind are bit flip errors and phase flip errors. When a bit flip error occurs a qubit $|\psi\rangle=\alpha|0\rangle+\beta|1\rangle$ becomes $|\psi\rangle=\beta|0\rangle+\alpha|1\rangle$. In a phase flip error, the relative sign between $\alpha$ and $\beta$ switches, i.e. $|\psi\rangle=\alpha|0\rangle+\beta|1\rangle$ becomes $|\psi\rangle=\alpha|0\rangle-\beta|1\rangle$.

First, we will discuss how to correct these basic errors separately and then together in a combined scheme. Later, we will show that any arbitrary qubit error can be converted into a bit flip or a phase flip error and therefore, correcting these two types of errors suffices for all purposes. This is quite a surprising result because although there are infinite number of errors possible, they can be discretized to just two types of simple errors.

### 08.01.03: Unique Challenges in Quantum Error Correction

While thinking in terms of bit-flip and phase-flip errors in nice, correcting these errors is not as straight forward. Features that we saw help enable quantum cryptography become the main hurdles here.

1. No-cloning: The simplest method of implementing error correction with classical bits is repetition codes. Errors can then be corrected using a majority voting mechanism. These work for classical bits because a classical bit can be read and copied as many times as we want. Qubits on the other hand are destroyed (collapse) once they are read (measured). Further, measurement of a single qubit only reveals a bit 0 or a bit 1 and never the actual values of $\alpha$ and $\beta$. Since one cannot determine the values of $\alpha$ and $\beta$ from a single qubit, it cannot be copied and repetition codes fail.

2. Phase-flip error: This has no analogue in classical error correction and therefore not as easy to construct a correction mechanism for.

3. Arbitrary errors: Bit-flip and phase-flip errors are "large" errors leading to a completely opposite state. Errors in qubits, however, can be small and perturb $\alpha$ and $\beta$ values only slightly by $\epsilon$.

4. No-measurement allowed: Not only that no-cloning theorem prohibits copying of qubits, one cannot make any kind of measurement on the original qubit you want to protect from errors. This will collapse the qubit and original information contained in it can never be recovered.

### 08.01.04: Correcting the Quantum Bit-flip Error

Consider a single qubit in state $|0\rangle$ or $|1\rangle$. To implement a repetition code for these qubits, we would like to map the qubits to logical qubits as shown below,

$$|0\rangle\rightarrow|000\rangle$$
and
$$|1\rangle\rightarrow|111\rangle$$

Therefore, given an arbitrary qubit $|\psi\rangle=\alpha|0\rangle+\beta|1\rangle$ we can introduce redundancy in the system (quite similar to classical error correction codes) by simply entangling the qubits with two other ancillary qubits using a CNOT gate. This will result in the logical mapping we desire.

$$|\psi\rangle\rightarrow\alpha|000\rangle+\beta|111\rangle$$

We'll denote this new logical qubit as $|\psi_l\rangle$

Assuming at most one bit-flip error occurs during transmission the final received logical qubit $|\psi'_l\rangle$ will be in one of the following three states,

\begin{aligned}
\alpha|100\rangle&+\beta|011\rangle \text{ or} \\
\alpha|010\rangle&+\beta|101\rangle \text{ or} \\
\alpha|001\rangle&+\beta|110\rangle
\end{aligned}

Now three parities can be checked (1,2), (2,3) and (1,3), i.e. between qubits 1 and 2, 2 and 3 and 1 and 3. Since at most one qubit is assumed to be in error two of these parity checks are enough. Our main goal is to distinguish which qubit is different. For example if we check (1,2) and (2,3) and the first parity check results in a 1 and the second in a 0, then we know that the first qubit is different from the second qubit and the second and third qubits are identical. Recall that CNOT is equivalent to an XOR operation and is used to implement these parity checks. However, these XOR operations cannot be done in place and we need a "blank slate" to write the results on. Therefore, the receiver uses two additional ancillary qubits to determine the error syndrome.

<b>Example:</b> Let the received logical qubit be $\alpha|100\rangle+\beta|011\rangle$. We use two ancillary qubits both in the state $|0\rangle$. Therefore the combined system state is given by $\alpha|1_10_20_30_40_5\rangle+\beta|0_11_21_30_40_5\rangle$. Subscripts are shown here for easy tracking of qubits.

We can now apply a CNOT between the first and the fourth qubits and then between the second and the fourth qubits. The final state of the fourth qubit after the two CNOT operations will be 1.

Similarly, CNOT is now applied between the second and the fifth qubit and then between third and the fifth qubit. The final state of the fifth qubit will be 0.

The new ancilla qubits are still unentangled and can be measured directly in computational basis without affecting the logical qubit. The error syndrome in the above example is 10 indicating that the first qubit is in error. This can be fixed by simply applying $X$ gate to the qubit 1 resulting in $|\psi\rangle$. Similarly, all three possibilities for bit flip errors can be corrected using the $X$ gate.

Once the original logical qubit is retrieved, CNOT operations between the first and second and first and third qubits will unentangle the three qubits giving us the corrected state $|\psi\rangle$.

### 08.01.05: Correcting the Quantum Phase-flip Error

Interestingly phase-flip errors can be converted to bit-flip errors by a simple change of basis and we already know how to correct bit-flip errors. Recall that:

\begin{aligned}
|+\rangle &= H|0\rangle=\frac{1}{\sqrt{2}}|0\rangle+|1\rangle \\
|-\rangle &= H|1\rangle=\frac{1}{\sqrt{2}}|0\rangle-|1\rangle \\
\end{aligned}

Therefore, in the Hadamard basis a phase-flip error transforms $|+\rangle$ to $|-\rangle$ and visa versa.

A simple trick to protect against phase-flip errors then is to transmit the qubits in Hadamard basis. Using the above formulation, if we want to transmit a qubit $|\psi\rangle=\alpha|0\rangle+\beta|1\rangle$, we map it to a logical qubit $|\psi_l\rangle=\alpha|000\rangle+\beta|111\rangle$ as before. Then we apply Hadamard gate, $H$, to all the qubits transforming the logical qubit into $|\psi_l\rangle=\alpha|+++\rangle+\beta|---\rangle$.

Now a phase-flip error will have three possibilities as before,

\begin{aligned}
\alpha|-++\rangle&+\beta|+--\rangle \text{ or} \\
\alpha|+-+\rangle&+\beta|-+-\rangle \text{ or} \\
\alpha|++-\rangle&+\beta|--+\rangle \\
\end{aligned}

When a Hadamard operation is applied to these qubits at the receiving end, they will transform to the same three possibilities that existed for the bit-flip error before and can be corrected as before.

### 08.01.06: Combining the Phase-flip and Bit-flip Error Correction Codes

We see that a three-qubit code can correct either a bit-flip or a phase-flip error. What if there is a chance of both errors happening? We can correct these with a 9-qubit error correction code.

The first is to encode the qubits as we did in phase-flip error correction codes. Therefore, $|0\rangle$ is mapped to $|+++\rangle$ and $|1\rangle$ is mapped to $|---\rangle$. In general we obtain $|\psi_1\rangle=\alpha|+++\rangle+\beta|---\rangle$.

In the next step we will encode the qubits for bit-flip error correction. Recall that $\alpha|0\rangle+\beta|1\rangle$ is encoded as $\alpha|000\rangle+\beta|111\rangle$. Also, recall that $|+\rangle=\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$. Therefore, $|+\rangle$ is encoded as $\frac{1}{\sqrt{2}}(|000\rangle+|111\rangle)$ and $|-\rangle$ is encoded as $\frac{1}{\sqrt{2}}(|000\rangle-|111\rangle)$. Therefore, the resulting $|\psi_2\rangle$ that is transmitted becomes:

$$\frac{\alpha}{2\sqrt{2}}(|000\rangle+|111\rangle)(|000\rangle+|111\rangle)(|000\rangle+|111\rangle)+\frac{\beta}{2\sqrt{2}}(|000\rangle-|111\rangle)(|000\rangle-|111\rangle)(|000\rangle-|111\rangle)$$

Here we have three groups of three qubits. In order to check for a bit flip error we had to do two parity checks, namely (1,2) and (2,3). Here we will follow the same logic and do six parity checks. These will be between qubits (1,2), (2,3), (4,5), (5,6), and (7,8) and (8,9). Correcting any errors found follows the same procedure as before. Further, we would need six ancilla qubits for these parity checks. Once the bit-flip error correction is done, one can simply apply CNOT operations and return the received state $|\psi_2\rangle$ to $|\psi_1\rangle$.

Now, we have reduced our system to the one similar to phase-flip error correction case discussed above and the same procedure can be applied to correct any phase-flips that may have happened.

Therefore, Shor's 9-qubit error correction code can correct bit-flip, phase-flip as well as a combination of bit-flip and phase-flip errors. However, the power of Shor's code goes much beyond this and can correct any arbitrary qubit error as our following analysis will show.

### 08.01.07: Correcting Arbitrary Errors

We mentioned before that any arbitrary quantum error can be converted to either bit-flip or phase-flip error. This is wonderful because we already know who to correct the latter two types of errors. Let's look at the details of why we say such a conversion is possible.

Recall the Pauli matrices that we discussed before. These are shown below,

$${\begin{aligned}X=\sigma _{x}&={\begin{pmatrix}0&1\\1&0\end{pmatrix}}\\Y=\sigma _{y}&={\begin{pmatrix}0&-i\\i&0\end{pmatrix}}\\Z=\sigma _{z}&={\begin{pmatrix}1&0\\0&-1\end{pmatrix}}\,\end{aligned}}$$

Pauli matrices along with the identity matrix can be used to represent any $2\times2$ arbitrary matrix. This coupled with the fact that arbitrary single qubit error can be described by a single qubit operation $U=\begin{pmatrix}u_{11}&u_{12}\\u_{21}&u_{22}\end{pmatrix}$, we see that we can decompose $U$ in terms of Pauli matrices.

Note that Pauli matrix $X$ is a bit-flip matrix, $Z$ is a phase-flip matrix and $ZX=iY$ is the combination of the two.

Now it is easy to see that $I, X, Z$ and $XZ$ are linearly independent and span the $2\times2$ matrix space. Therefore, an arbitrary single qubit operation $U$ can be written as:

$$U=\begin{pmatrix}u_{11}&u_{12}\\u_{21}&u_{22}\end{pmatrix}=\frac{u_{11}+u_{12}}{2}I +\frac{u_{12}+u_{21}}{2}X +\frac{u_{11}-u_{22}}{2}Z +\frac{u_{12}-u_{21}}{2}ZX$$

Therefore, if the error matrix $U$ acts on a state $|\psi\rangle$ then the new state:

$$|\psi'\rangle=U|\psi\rangle=\frac{u_{11}+u_{12}}{2}I|\psi\rangle +\frac{u_{12}+u_{21}}{2}X|\psi\rangle +\frac{u_{11}-u_{22}}{2}Z|\psi\rangle +\frac{u_{12}-u_{21}}{2}ZX|\psi\rangle$$

Upon measuring the error syndrome the superposition will collapse to one of the four possibilities and can be corrected by the application of gates X, Z or ZX or by doing nothing in the case of no errors.

## 08.02: Attacks on QKD Implementations

### 08.02.01: Attacks on QKD Implementations

Quantum channels connecting Alice and Bob often suffer from absorption and scattering due to which not all the states that Alice transmits reach Bob. In addition, even the ones that do make it to Bob may be in error due to noise in the channel and therefore result in an incorrect bit. Although some of these errors may be caused due to Eve, not all are due to Eve.

Similar to non-ideal channels, we also have other non-ideal components such as photon sources and photon detectors. As mentioned before, most common implementations use attenuated lasers to implement single photon sources. The photons emitted exhibit a Poisson probability distribution where the mean photon number is given by $\mu$. Typically $\mu<1$ is employed which means most of the time periods do not even contain a photon! On the other hand, some of the time periods contain more than one photon.

Similar to imperfect photon emitters, there are imperfect photon detectors that may suffer from dark noise (dark clicks). In other words, the photon detector may sometimes click even when there is no photon present!

In addition to hardware imperfections, other factors such as optically misaligned components may lead to errors in detection as well as stray light can lead to false clicks.

Since the procedure to detect Eve basically looks for errors in the received bit sequence, Eve is able to successfully launch an attack if she can use these imperfections to her advantage. This in means she needs to stay below the abort threshold of quantum bit error rate (QBER) and channel transmittance that Alice and Bob expect. Several manufacturers of quantum equipment such as IDQuantique abort protocols at around 8% error rates and about 0.15 channel transmittance.

#### 1. Photon number splitting attack

The photon number splitting attack, or the PNS attack for short, is the simplest attack to understand conceptually and also reemphasizes the importance of single photon emitters. The attack assumes that Eve has a photon counting device and if the photon emitter produces more than one photon in a given pulse Eve will siphon off the excess photons and let one pass through to Bob. Alice and Bob would suspect absolutely nothing since this attack does not effect the quantum bit error rate (QBER).

In the context of BB84, if there are two photons in a pulse, then Eve can siphon of one of them and measure it in a randomly chosen basis. With a 50% probability she will share that bit with Alice and Bob.

On the other hand, if there are three (or more) photons in a pulse, Eve really just needs two of the photons that she will measure in different bases and therefore record both possible outcomes. During the public discussion phase she will simply discard the outcome that resulted from the incorrect bases measurement. Eve will therefore have the correct bit value for such multi-photon pulses.

#### 2. Intercept and resend attack

Intercept and resend is the most naive attack Eve can launch. In this case, she simply blocks all the pulses, measures them randomly and reconstructs these pulses based on her observation and sends them off. In intercept and resend attack she will cause about 50% of the qubits that were supposed to be received correctly collapse randomly. Therefore, her presence will be detected with a probability of 25%.

Eve can, on the other hand, decide to only intercept and resend a fraction of the qubits in the hope that the amount of error that she causes remains well below the channel quantum bit error rate. In this case, privacy amplification should eliminate any partial information that Eve was able to determine.

#### 3. Denial of service attack

This follows directly from the intercept and resend attack. If Eve naively decides to measure all the qubits such that she is detected then Alice and Bob will either have to move to a different quantum channel or wait for sometime before trying again. If Eve constantly interferes with the qubits, Alice and Bob will never be able to communicate and therefore become victims of a denial of service attack.

#### 4. Man-in-the-middle attack

A man-in-the-middle attack may be possible if there is no classical authenticated channel between Alice and Bob. In this case, Eve can simply do an intercept and resend attack on the quantum channel and then launch a conventional man-in-the-middle attack on the classical channel without either Bob or Alice suspecting anything and each thinking that they are talking to the legitimate party. Therefore, an authenticated classical channel is a fundamental requirement.

One can therefore argue that quantum key distribution schemes really are just a way to expand a small amount of authentication information into a large encryption key.

#### 5. Trojan attack

This is a somewhat sophisticated attack in that it can reveal the entire encryption key without leaving a trace. The idea in this attack is to send bright pulses into Alice and Bob's equipment and observing the back reflection that occurs. It turns out that this back reflection reveals the exact orientation of Alice and Bob's modulators and thus the secret basis they are using. Eve can then simply set her basis to the same basis and measure the qubit. This attack has been demonstrated against Calvis2 (IDQuantique) equipment.

#### 6. Faked-state attack

Several of the recent attacks involve *faked states* of light which are specially crafted optical signals that Eve prepares and sends to Bob in order to control Bob's detector in a manner that she wishes. The goal of Eve here is not to read the original qubits correctly but to make Bob only read qubits of her choosing such that in the end only the qubits where all the three bases between Alice, Bob and Eve match up are retained. In order to achieve this Eve shines a bright light at Bob's detector forcing Bob's detector to operate in linear mode acting as a simple avalanche photo diode.

Faked state attack is typically implemented as a intercept and resend attack. Therefore, Eve intercepts Alice's photon and measures it in randomly chosen basis. if Eve's and Alice's bases match, they will share a common bit $b_e=b_a$. Eve will now reconstruct a photon and transmit to Bob. Here, due to the linear mode of operation of Bob's detector two possible cases may happen.

a. Bob and Eve measure in the same basis leading to a click at Bob's detector.
    
b. Bob and Eve's bases differ in which case no click takes place at Bob's end.

If on the other hand, if Alice's and Eve's bases don't match then Eve's signal is detected at Bob's end only when Alice's and Bob's bases don't match. These bits will eventually be thrown out during the key sifting phase.

Therefore, after key sifting only bits where all three bases coincide would be retained and remaining thrown away. Eve would therefore have the entire key. She can now simply listen to the classical error correction protocol for information reconciliation and privacy amplification.

The most common method for implementing faked state attack is blinding attack where the detector falls into the linear mode of operation and thus becomes "blind" to single photon pulses.